In [ ]:
import pandas as pd
import math
from collections import defaultdict


#---------------------------------------------------------------
# 1) Read CSV files
#---------------------------------------------------------------
cart_df      = pd.read_csv("cart.csv")       # CART_ID, INIT_LOC
wip_df       = pd.read_csv("wip.csv")        # WIP_ID, QTIME, FROM, TO
xfer_time_df = pd.read_csv("xfer.csv")  # FROM_LOC, TO_LOC, TIME

#---------------------------------------------------------------
# 2) Convert to Python data structures
#---------------------------------------------------------------
CARTS = cart_df.to_dict(orient="records")
WIPS  = wip_df.to_dict(orient="records")

# XFER_TIME as dict of dict, for quick lookup
# e.g. XFER_TIME[A][B] = travel time from A->B
XFER_TIME = defaultdict(dict)
for _, row in xfer_time_df.iterrows():
    A = row['FROM']
    B = row['TO']
    t = row['XFER_TIME']
    XFER_TIME[A][B] = t

def get_travel_time(locA, locB):
    """Return the travel time from locA to locB (default large if missing)."""
    return XFER_TIME[locA].get(locB, math.inf)

#---------------------------------------------------------------
# 3) Initialize cart states
#   cart_state[cid] = {'current_loc': <str>, 'available_time': <float>}
#---------------------------------------------------------------
cart_state = {}
for cart in CARTS:
    cid = cart['CART_ID']
    cart_state[cid] = {
        'current_loc': cart['INIT_LOC'],
        'available_time': 0.0
    }

# Keep track of actions count per cart
order_counter = defaultdict(int)

#---------------------------------------------------------------
# 4) Sort WIPs by ascending Q-Time
#---------------------------------------------------------------
WIPS_sorted = sorted(WIPS, key=lambda w: w['Remaining Q-Time'])

# Track which WIPs are scheduled or expired
scheduled = set()     # WIP_IDs that got scheduled
expired_wips = []     # WIPs that couldn't meet Q-Time

# Prepare final schedule
schedule = []

total_distance = 0

#---------------------------------------------------------------
# 5) Define functions to simulate single or double WIP trips
#---------------------------------------------------------------

def simulate_single_trip(cid, state, wip):
    """
    Return tuple (feasible, distance, finish_time, timeline)
    where timeline is a list of (ACTION, COMPLETE_TIME, WIP_ID).
    """
    wip_id   = wip['WIP_ID']
    qtime    = wip['Remaining Q-Time']
    from_loc = wip['FROM']
    to_loc   = wip['TO']

    start_time = state['available_time']
    cart_loc   = state['current_loc']

    # Times
    t1 = get_travel_time(cart_loc, from_loc)  # cart -> pickup
    t2 = 1.0  # pickup duration
    t3 = get_travel_time(from_loc, to_loc)    # pickup -> delivery

    arrival_pickup = start_time + t1
    pickup_done    = arrival_pickup + t2
    deliver_done   = pickup_done + t3

    # Check Q-Time
    if deliver_done > qtime:
        return (False, math.inf, deliver_done, [])

    # If feasible, build timeline
    distance = t1 + t3
    timeline = [
        ("PICKUP",   pickup_done,  wip_id),
        ("DELIVERY", deliver_done, wip_id),
    ]
    return (True, distance, deliver_done, timeline)

def simulate_two_wips(cid, state, wipA, wipB, mode):
    """
    Simulate 2-WIP trip for cart `cid` in either Mode A or B:
        Mode A: PICKUP(A)–DELIVERY(A)–PICKUP(B)–DELIVERY(B)
        Mode B: PICKUP(A)–PICKUP(B)–DELIVERY(A)–DELIVERY(B)

    Return (feasible, distance, finish_time, timeline).
    If feasible = False, ignore other return values.
    """
    # Unpack WIPs
    A_id   = wipA['WIP_ID']
    A_qt   = wipA['Remaining Q-Time']
    A_from = wipA['FROM']
    A_to   = wipA['TO']

    B_id   = wipB['WIP_ID']
    B_qt   = wipB['Remaining Q-Time']
    B_from = wipB['FROM']
    B_to   = wipB['TO']

    start_time = state['available_time']
    cart_loc   = state['current_loc']

    # We'll track a timeline and total distance
    timeline = []
    distance = 0.0
    current_time = start_time
    current_loc  = cart_loc

    # Helper function to "move & pick up"
    def do_pickup(wip_id, from_loc):
        nonlocal current_time, current_loc, distance, timeline
        ttravel = get_travel_time(current_loc, from_loc)
        current_time += ttravel
        distance += ttravel
        # pickup time
        tpick = 1.0
        current_time += tpick
        timeline.append(("PICKUP", current_time, wip_id))
        current_loc = from_loc

    # Helper function to "deliver"
    def do_deliver(wip_id, to_loc):
        nonlocal current_time, current_loc, distance, timeline
        ttravel = get_travel_time(current_loc, to_loc)
        current_time += ttravel
        distance += ttravel
        timeline.append(("DELIVERY", current_time, wip_id))
        current_loc = to_loc

    # Mode A: A->Delivery, then B->Delivery
    if mode == "A":
        # Pickup A
        do_pickup(A_id, A_from)
        # Deliver A
        do_deliver(A_id, A_to)
        # Check A Q-Time violation
        if timeline[-1][1] > A_qt:  # last action time for A
            return (False, math.inf, current_time, [])

        # Pickup B
        do_pickup(B_id, B_from)
        # Deliver B
        do_deliver(B_id, B_to)
        # Check B Q-Time violation
        if timeline[-1][1] > B_qt:
            return (False, math.inf, current_time, [])

    # Mode B: Pickup A, Pickup B, then Deliver A, Deliver B
    elif mode == "B":
        # Pickup A
        do_pickup(A_id, A_from)
        # Pickup B
        do_pickup(B_id, B_from)
        # Deliver A
        do_deliver(A_id, A_to)
        if timeline[-1][1] > A_qt:
            return (False, math.inf, current_time, [])
        # Deliver B
        do_deliver(B_id, B_to)
        if timeline[-1][1] > B_qt:
            return (False, math.inf, current_time, [])
    else:
        raise ValueError("Mode must be 'A' or 'B'.")

    # If we get here, no Q-Time violation
    return (True, distance, current_time, timeline)

#---------------------------------------------------------------
# 6) Main loop: try to assign WIPs, attempt pairing
#---------------------------------------------------------------
# iterate over WIPs in ascending Q-Time, but won't skip if can pair it with another.
# So need a while loop with an index to handle possible pairing.
i = 0
n = len(WIPS_sorted)

while i < n:
    wipA = WIPS_sorted[i]
    wipA_id = wipA['WIP_ID']

    # If already scheduled, skip
    if wipA_id in scheduled:
        i += 1
        continue

    best_cart = None
    best_distance = math.inf
    best_timeline = []
    best_finish_time = 0.0
    best_mode = None
    # We'll keep track if we used two WIPs or single
    best_is_two = False
    partner_idx = None

    #-----------------------------------------------------------
    # (a) Try single scheduling for wipA on each cart
    #-----------------------------------------------------------
    for cid, state in cart_state.items():
        feasible, dist, finish_time, timeline = simulate_single_trip(cid, state, wipA)
        if feasible and dist < best_distance:
            best_distance   = dist
            best_cart       = cid
            best_timeline   = timeline
            best_finish_time= finish_time
            best_mode       = "SINGLE"
            best_is_two     = False
            partner_idx     = None

    #-----------------------------------------------------------
    # (b) Try pairing wipA with another unscheduled wipB
    #     will pick wipB from the *remaining* unscheduled WIPs.
    #-----------------------------------------------------------
    for j in range(i+1, n):
        wipB = WIPS_sorted[j]
        wipB_id = wipB['WIP_ID']
        if wipB_id in scheduled:
            continue

        # Try each cart in modes A/B
        for cid, state in cart_state.items():
            # Mode A
            feasibleA, distA, finishA, timelineA = simulate_two_wips(cid, state, wipA, wipB, "A")
            # Mode B
            feasibleB, distB, finishB, timelineB = simulate_two_wips(cid, state, wipA, wipB, "B")

            # Evaluate whichever is feasible & best
            # will pick the best among feasibleA, feasibleB
            chosen_feasible = False
            chosen_dist = math.inf
            chosen_finish = 0
            chosen_timeline = []
            chosen_mode = None

            if feasibleA:
                chosen_feasible = True
                chosen_dist = distA
                chosen_finish = finishA
                chosen_timeline = timelineA
                chosen_mode = "A"

            if feasibleB and distB < chosen_dist:
                chosen_feasible = True
                chosen_dist     = distB
                chosen_finish   = finishB
                chosen_timeline = timelineB
                chosen_mode     = "B"

            # If found a feasible pairing that’s better than our current best
            if chosen_feasible and chosen_dist < best_distance:
                best_cart = cid
                best_distance   = chosen_dist
                best_finish_time= chosen_finish
                best_timeline   = chosen_timeline
                best_mode       = chosen_mode
                best_is_two     = True
                partner_idx     = j

    #-----------------------------------------------------------
    # 7) Check if found any feasible assignment for wipA
    #-----------------------------------------------------------
    if not best_cart:
        # No feasible single or paired => wipA expires
        expired_wips.append(wipA_id)
        i += 1
        continue

    #-----------------------------------------------------------
    # 8) have a best assignment: either single or double
    #    Update schedule, cart state, etc.
    #-----------------------------------------------------------
    chosen_cart_state = cart_state[best_cart]
    start_time = chosen_cart_state['available_time']
    start_loc  = chosen_cart_state['current_loc']

    # "Apply" best_timeline
    # will keep track of how the cart's time & loc evolves
    current_time = start_time
    current_loc  = start_loc

    # Add lines in schedule
    for (action, ctime, wip_id) in best_timeline:
        # assume distance was accounted for above
        # But need to figure out how many minutes we traveled to get from
        # current_loc to next action loc.

        order_counter[best_cart] += 1
        schedule.append({
            'CART_ID': best_cart,
            'ORDER': order_counter[best_cart],
            'WIP_ID': wip_id,
            'ACTION': action,
            'COMPLETE_TIME': ctime
        })
        current_time = ctime
        # If action=DELIVERY, the location is wip's destination
        # If action=PICKUP, the location is wip's FROM
        # But in the timeline creation, we already track that,
        # so let’s do a simpler approach: we’ll parse from timeline if needed.
        # For clarity, we won't do location updates here because we stored them in the timeline function.

    # We also increment total_distance by best_distance
    total_distance += best_distance

    # The final location in timeline is the location of the last action
    # We can parse best_timeline[-1], but we also already returned best_finish_time
    # We'll reconstruct the final "delivery" location:
    # If best_mode=SINGLE => wipA, to_loc
    # If best_mode in [A,B] => last was wipB's to_loc

    if best_is_two:
        # We used wipA + wipB
        wipB = WIPS_sorted[partner_idx]
        wipB_id = wipB['WIP_ID']
        scheduled.add(wipA_id)
        scheduled.add(wipB_id)
        # Let’s figure out that last delivered WIP’s "TO" location
        # The timeline’s last entry is ("DELIVERY", time, wipX_id)
        # We can do:
        last_action = best_timeline[-1]
        last_wip_id = last_action[2]
        final_loc = next(x['TO'] for x in [wipA, wipB] if x['WIP_ID'] == last_wip_id)

        chosen_cart_state['current_loc']    = final_loc
        chosen_cart_state['available_time'] = best_finish_time

        # Mark wipB as consumed => we skip it in future
        # We'll also skip the index j (partner_idx)
        # Because we’re about to do i += 1 below, we can set wipB’s scheduled
        # But to properly skip in the loop, we might set partner_idx’s WIP to scheduled
        # so in a real system, we either remove it or note it. We do:
        # Done above by `scheduled.add(wipB_id)`

    else:
        # Single WIP
        scheduled.add(wipA_id)
        final_loc = wipA['TO']
        chosen_cart_state['current_loc']    = final_loc
        chosen_cart_state['available_time'] = best_finish_time

    # If it was a two-WIP scenario, we skip the second WIP in the main loop:
    if best_is_two and partner_idx is not None:
        # We know wipB is also scheduled, so let's not try to schedule it again
        # If partner_idx < i => it's in the past
        # If partner_idx > i => we can mark it so we skip it
        # easiest is just to do: scheduled.add(WIPS_sorted[partner_idx]['WIP_ID'])
        # which we already did above. 
        pass

    # Finally, move i forward
    i += 1

#---------------------------------------------------------------
# 9) Print final schedule
#---------------------------------------------------------------
# Sort by CART_ID, ORDER
def cart_order_key(row):
    return (row['CART_ID'], row['ORDER'])

schedule_sorted = sorted(schedule, key=cart_order_key)

print("\n=== FINAL SCHEDULE ===")
print("CART_ID | ORDER | WIP_ID | ACTION    | COMPLETE_TIME")
for row in schedule_sorted:
    print(f"{row['CART_ID']:>6} | {row['ORDER']:>5} | {row['WIP_ID']:>5} | "
            f"{row['ACTION']:>8} | {row['COMPLETE_TIME']:>13.1f}")

#---------------------------------------------------------------
# 10) Print expired WIPs
#---------------------------------------------------------------
if expired_wips:
    print("\nExpired WIPs (could not meet Q-Time):")
    for wip_id in expired_wips:
        print(" -", wip_id)
else:
    print("\nNo expired WIPs!")

#---------------------------------------------------------------
# 11) Print total distance
#---------------------------------------------------------------
print(f"\nTotal distance traveled (all carts): {total_distance:.1f}")

#---------------------------------------------------------------
# 12) Write schedule to CSV
#---------------------------------------------------------------
schedule_df = pd.DataFrame(schedule_sorted)
schedule_df.to_csv("SCHEDULE.csv", index=False)




=== FINAL SCHEDULE ===
CART_ID | ORDER | WIP_ID | ACTION    | COMPLETE_TIME
   C01 |     1 |   W29 |   PICKUP |           3.0
   C01 |     2 |   W29 | DELIVERY |          24.0
   C01 |     3 |   W08 |   PICKUP |          30.0
   C01 |     4 |   W08 | DELIVERY |          56.0
   C01 |     5 |   W18 |   PICKUP |          59.0
   C01 |     6 |   W18 | DELIVERY |          64.0
   C01 |     7 |   W19 |   PICKUP |          73.0
   C01 |     8 |   W19 | DELIVERY |          78.0
   C02 |     1 |   W02 |   PICKUP |          14.0
   C02 |     2 |   W02 | DELIVERY |          31.0
   C02 |     3 |   W38 |   PICKUP |          32.0
   C02 |     4 |   W38 | DELIVERY |          50.0
   C02 |     5 |   W35 |   PICKUP |          53.0
   C02 |     6 |   W35 | DELIVERY |          81.0
   C03 |     1 |   W11 |   PICKUP |          34.0
   C03 |     2 |   W11 | DELIVERY |          41.0
   C03 |     3 |   W26 |   PICKUP |          47.0
   C03 |     4 |   W26 | DELIVERY |          50.0
   C03 |     5 |   W21 